In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
dataset=pd.read_csv('PP_Mtrls_and_their_Mchncl_Prprts.csv')
dataset.head()

,Material,Su,Sy,E,G,mu,Ro,Use
0,ANSI Steel SAE 1015 asrolled,421,314,207000,79000,0.3,7860,True
1,ANSI Steel SAE 1015 normalized,424,324,207000,79000,0.3,7860,True
2,ANSI Steel SAE 1015 annealed,386,284,207000,79000,0.3,7860,True
3,ANSI Steel SAE 1020 asrolled,448,331,207000,79000,0.3,7860,True
4,ANSI Steel SAE 1020 normalized,441,346,207000,79000,0.3,7860,True


In [3]:
dataset.drop('Material', axis=1, inplace=True)

In [4]:
columns_to_drop = ['Sy','mu']
dataset = dataset.drop(columns=columns_to_drop)
dataset.head()

,Su,E,G,Ro,Use
0,421,207000,79000,7860,True
1,424,207000,79000,7860,True
2,386,207000,79000,7860,True
3,448,207000,79000,7860,True
4,441,207000,79000,7860,True


In [5]:
dataset['Use'] = dataset['Use'].astype('object')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802 entries, 0 to 801
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Su      802 non-null    int64 
 1   E       802 non-null    int64 
 2   G       802 non-null    int64 
 3   Ro      802 non-null    int64 
 4   Use     802 non-null    object
dtypes: int64(4), object(1)
memory usage: 31.5+ KB


In [6]:
dataset=pd.get_dummies(dataset,drop_first=True)

dataset.head()

,Su,E,G,Ro,Use_True
0,421,207000,79000,7860,True
1,424,207000,79000,7860,True
2,386,207000,79000,7860,True
3,448,207000,79000,7860,True
4,441,207000,79000,7860,True


In [7]:
label_encoder = LabelEncoder()

# Fit and transform the target column
dataset['Use_True'] = label_encoder.fit_transform(dataset['Use_True'])
dataset

,Su,E,G,Ro,Use_True
0,421,207000,79000,7860,1
1,424,207000,79000,7860,1
2,386,207000,79000,7860,1
3,448,207000,79000,7860,1
4,441,207000,79000,7860,1
...,...,...,...,...,...
797,980,206000,80000,7860,0
798,1100,206000,80000,7860,0
799,1200,206000,80000,7860,0
800,1030,206000,80000,7860,0


In [8]:
indep=dataset[['Su','E','G','Ro']]
dep=dataset["Use_True"]

In [9]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
from sklearn.neighbors import KNeighborsClassifier

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors':[1,50],'p':[1,2],'weights':['uniform', 'distance'],'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
              'metric': ['str','minkowski']} 
grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1') 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 64 candidates, totalling 320 fits


/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
160 fits failed out of a total of 320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/neighbors/_classification.py", line 213, in fit
    self._validate_params()
  File "/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/

GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['str', 'minkowski'], 'n_neighbors': [1, 50],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             scoring='f1', verbose=3)

In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[250   5]
 [  4   9]]


In [14]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       255
           1       0.64      0.69      0.67        13

    accuracy                           0.97       268
   macro avg       0.81      0.84      0.82       268
weighted avg       0.97      0.97      0.97       268



In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 50, 'p': 2, 'weights': 'distance'}: 0.9670068126948752


In [16]:
from sklearn.metrics import roc_auc_score
roc_auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])
print("The roc_auc_score is:",roc_auc_score)

The roc_auc_score is: 0.9895927601809955


In [17]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000304,0.000121,0.000000,0.000000,auto,str,1,1,uniform,"{'algorithm': 'auto', 'metric': 'str', 'n_neig...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,33
1,0.000383,0.000323,0.000000,0.000000,auto,str,1,1,distance,"{'algorithm': 'auto', 'metric': 'str', 'n_neig...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,33
2,0.001507,0.002567,0.000000,0.000000,auto,str,1,2,uniform,"{'algorithm': 'auto', 'metric': 'str', 'n_neig...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,33
3,0.000100,0.000013,0.000000,0.000000,auto,str,1,2,distance,"{'algorithm': 'auto', 'metric': 'str', 'n_neig...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,33
4,0.000198,0.000102,0.000000,0.000000,auto,str,50,1,uniform,"{'algorithm': 'auto', 'metric': 'str', 'n_neig...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.000588,0.000329,0.002963,0.001549,brute,minkowski,1,2,distance,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.250000,0.727273,0.800000,0.666667,0.666667,0.622121,0.192427,7
60,0.000652,0.000493,0.004837,0.000508,brute,minkowski,50,1,uniform,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25
61,0.000469,0.000239,0.003063,0.001143,brute,minkowski,50,1,distance,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.500000,0.600000,0.666667,0.666667,0.666667,0.620000,0.065320,9
62,0.000430,0.000289,0.004280,0.000402,brute,minkowski,50,2,uniform,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25


In [20]:
su_input=float(input("Ultimate Tensile Strength (Su) in MPa:"))
E_input=float(input("Elastic Modulus (E) in MPa:"))
g_input=float(input("Shear Modulus (G) in MPa:"))
ro_input=float(input("Density (Ro) in Kg/m3:"))

Ultimate Tensile Strength (Su) in MPa:1000
Elastic Modulus (E) in MPa:119876
Shear Modulus (G) in MPa:67543
Density (Ro) in Kg/m3:8912


In [21]:
Future_Prediction=grid.predict([[su_input,E_input,g_input,ro_input]])
y=("Future_Prediction={}".format(Future_Prediction))
if (y==0):
    print("Future_Prediction=[False]")
else:
    print('Future_Prediction=[True]')

Future_Prediction=[True]
